<a href="https://colab.research.google.com/github/LC1332/Embed-Adapter/blob/main/notebook/maximul_batch_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 从utils载入batch测试
- [x] 构造大量长度超过512的文本
- [x] 在A100下进行batch测试

我希望为我的中文sentence embedding做显存极限测试

请先为我生成一个list of str， seed_strs ，里面有8句左右的短小的中文，

然后循环生成一个texts,texts中共有200个元素，每个text为seed_strs随机拼接而成，并且长度都超过1000，请用python为我实现

In [1]:
seed_strs = [
    "今天的天气真好。",
    "我喜欢阅读历史书籍。",
    "猫是非常可爱的动物。",
    "科技正在改变我们的生活。",
    "音乐能够治愈人的心灵。",
    "旅行是一种了解世界的方式。",
    "美食总是令人心情愉快。",
    "运动对健康非常重要。"
]

import random

texts = []
for _ in range(200):
    text = ""
    while len(text) < 1000:
        text += random.choice(seed_strs)
    texts.append(text)

# 验证第一个元素的长度
print(len(texts[0]))


1006


In [2]:
%cd /content
!rm -rf /content/Embed-Adapter
!git clone https://github.com/LC1332/Embed-Adapter
%cd /content/Embed-Adapter

/content
Cloning into 'Embed-Adapter'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 71 (delta 31), reused 41 (delta 13), pack-reused 0
Receiving objects: 100% (71/71), 20.29 MiB | 18.89 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/Embed-Adapter


In [6]:
from EmbedAdapter.utils import get_general_embeddings

对embedding函数的测试

测试这两个

BAAI/bge-small-zh-v1.5

BAAI/bge-base-zh-v1.5

In [7]:
test_batch_n = 4

test_batch = texts[:test_batch_n]

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

In [17]:
import plotly.graph_objects as go
import numpy as np

# 给定的数据
batch_size = np.array([4, 8, 16, 32, 64, 128, 192, 256])
GPU_RAM = np.array([2.7, 3, 3.5, 4.5, 6.5, 10.5, 11.7, 13.9])

# 创建散点图
scatter = go.Scatter(x=batch_size, y=GPU_RAM, mode='markers', name='实际数据点')

# 创建趋势线
# 为了延伸到 batch_size = 512，我们使用 numpy 的 polyfit 和 poly1d 函数来拟合数据
p = np.poly1d(np.polyfit(batch_size, GPU_RAM, 1))
trend_x = np.linspace(4, 512, 100)  # 生成从4到512的100个点
trend_y = p(trend_x)

trend_line = go.Scatter(x=trend_x, y=trend_y, mode='lines', name='趋势线')

# 创建图表
fig = go.Figure(data=[scatter, trend_line])

# 设置图表的标题和坐标轴标签
fig.update_layout(title='Batch Size vs GPU RAM Usage',
                  xaxis_title='Batch Size',
                  yaxis_title='GPU RAM (GB)',
                  xaxis=dict(range=[0, 512]))  # 设置x轴的范围

# 显示图表
fig.show()


batch=4的时候 使用了2.7G

In [9]:
test_batch_n = 8

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




batch = 8的时候使用了3G

In [10]:
test_batch_n = 16

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




16的时候3.5G

In [11]:
test_batch_n = 32

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




32的时候4.5G

In [12]:
test_batch_n = 64

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




64的时候6.5G

In [13]:
test_batch_n = 128

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




128的时候10.5G

In [14]:
test_batch_n = 128+64

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




192的时候11.7G

In [15]:
test_batch_n = 256

test_batch = texts[:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-zh-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-zh-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-zh-v1.5", return_tensor = True)




256的时候13.9G

In [ ]:
batch_size = [4,8,16,32,64,128,192,256]
GPU_RAM = [2.7 , 3 ,3.5,4.5,6.5,10.5,11.7,13.9]

batch_size = [4,8,16,32,64,128,192,256]
GPU_RAM = [2.7 , 3 ,3.5,4.5,6.5,10.5,11.7,13.9]

请为我用plotly，画出横轴为batch_size, 纵轴为GPU_RAM的图像，同时画出散点和趋势

并向外延伸到batch_size = 512的情况